# Scrap the books (name, price, rate, category_name) for each category and put them into a CSV & Excel file & db file
https://books.toscrape.com/

Also expected csv attached.

In [14]:
from bs4 import BeautifulSoup
import requests
import csv
book_name = []
book_price = []
book_rate = []
book_ratenum = []
book_category = []
ratedict = {"One Stars":1,"Two Stars":2,"Three Stars":3,"Four Stars":4,"Five Stars":5}
x= 1

In [15]:
while True:
    url = f"https://books.toscrape.com/catalogue/page-{x}.html"
    x+=1
    response = requests.get(url)
    src = response.text
    soup = BeautifulSoup(src, "html")
    book_elements = soup.find_all("li", attrs= {"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    for book in book_elements:
        book_name.append(book.find("a").find("img").attrs["alt"])
        book_price.append(float(book.find('p', attrs={'class': 'price_color'}).get_text().split('£')[1]))
        book_rate.append(book.find("p").attrs["class"][1]+ " Stars")
        book_ratenum.append(ratedict[book.find("p").attrs["class"][1]+ " Stars"])
        book_link = "https://books.toscrape.com/catalogue/"+ book.find("h3").find("a").get("href")
        response2 = requests.get(book_link)
        categ = response2.text
        soup2 = BeautifulSoup(categ,"html.parser")
        book_category.append(soup2.find_all("li")[2].find("a").text)
    if x == 51:
        break
    print(f"Page number{x}")

Page number2
Page number3
Page number4
Page number5
Page number6
Page number7
Page number8
Page number9
Page number10
Page number11
Page number12
Page number13
Page number14
Page number15
Page number16
Page number17
Page number18
Page number19
Page number20
Page number21
Page number22
Page number23
Page number24
Page number25
Page number26
Page number27
Page number28
Page number29
Page number30
Page number31
Page number32
Page number33
Page number34
Page number35
Page number36
Page number37
Page number38
Page number39
Page number40
Page number41
Page number42
Page number43
Page number44
Page number45
Page number46
Page number47
Page number48
Page number49
Page number50


In [16]:
zip_list = list(zip(book_name,book_price,book_ratenum,book_category))

In [17]:
with open("books.csv","w",newline="",encoding = 'charmap') as b:
    writer = csv.writer(b)
    writer.writerow(["Book Name","Price","Rate","book_category"])
    writer.writerows(zip_list)

In [18]:
import pandas as pd
df = pd.read_csv("books.csv",encoding = 'unicode_escape')
df.to_excel('books.xlsx',index=False)

In [19]:
pd.read_csv("books.csv",encoding = 'unicode_escape')

,Book Name,Price,Rate,book_category
0,A Light in the Attic,51.77,3,Poetry
1,Tipping the Velvet,53.74,1,Historical Fiction
2,Soumission,50.10,1,Fiction
3,Sharp Objects,47.82,4,Mystery
4,Sapiens: A Brief History of Humankind,54.23,5,History
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,55.53,1,Classics
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,4,Sequential Art
997,A Spy's Devotion (The Regency Spies of London #1),16.97,5,Historical Fiction
998,1st to Die (Women's Murder Club #1),53.98,1,Mystery


In [20]:
import sqlite3
connection = sqlite3.connect("book.db")
connection.close()

In [21]:
book_table ="""CREATE TABLE "book" (
	"bookID"	INTEGER NOT NULL,
	"bookName"	NUMERIC NOT NULL,
	"bookRate"	INTEGER NOT NULL,
	"bookPrice"	INTEGER NOT NULL,
	"categoryName"	TEXT ,
	PRIMARY KEY("bookID" AUTOINCREMENT)
);"""


cat_table="""CREATE TABLE "category" (
	"categoryID"	INTEGER NOT NULL,
	"categoryName"	TEXT
);"""

In [22]:
connection = sqlite3.connect("book.db")
rows = connection.execute(book_table).fetchall()
connection.commit()
connection.close()

In [23]:
connection = sqlite3.connect("book.db")
rows = connection.execute(cat_table).fetchall()
connection.commit()
connection.close()

In [24]:
for i in range(len(book_category)):  
    connection = sqlite3.connect("book.db")
    cursor = connection.execute("INSERT INTO book (bookName,bookRate,bookPrice,categoryName) VALUES (?,?,?,?)",[book_name[i],book_ratenum[i],book_price[i],book_category[i]])
    connection.commit()
    connection.close()
    cursor.lastrowid
connection.close()

In [25]:
cat_ID = {key : value for key, value in zip(book_category , range(1,len(book_category) + 1))}

In [26]:
for i in range(len(book_category)):  
    connection = sqlite3.connect("book.db")
    cursor = connection.execute("INSERT INTO category (categoryID,categoryName) VALUES (?,?)",[cat_ID[book_category[i]],book_category[i]])
    connection.commit()
    connection.close()
    cursor.lastrowid
connection.close()